In [2]:
import numpy as np
import pylidc as pl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import math
import pandas as pd
import os
import cv2

In [29]:
scan_list = pl.query(pl.Scan).all()

In [4]:
def get_max_frame_size():
    height_max = width_max = 0

    for index, scan in enumerate(scan_list):
        print(f"At scan {index+1}/{len(scan_list)}")
        for ann in scan.annotations:
            bbox = ann.bbox()
            height, width, _ = ann.scan.to_volume(verbose=False)[bbox].shape
            height_max = max(height_max, height)
            width_max = max(width_max, width)
    
    # preventing from height_max and width_max being odd values
    if height_max%2 == 1: height_max += 1
    if width_max%2 == 1: width_max += 1
    return height_max, width_max

In [17]:
def get_zoomed_out_bbox(cur_bbox, i_range, j_range):
    i_middle = cur_bbox[0].start + (cur_bbox[0].stop - cur_bbox[0].start) // 2
    j_middle = cur_bbox[1].start + (cur_bbox[1].stop - cur_bbox[1].start) // 2
    
    padding_i = 0
    if i_middle - i_range//2 < 0:
        padding_i = abs(i_middle - i_range//2)
    elif i_middle + i_range//2 >= 512:
        padding_i = 511 - (i_middle + i_range//2)
    
    padding_j = 0
    if j_middle - j_range//2 < 0:
        padding_j = abs(j_middle - j_range//2)
    elif j_middle + j_range//2 >= 512:
        padding_j = 511 - (j_middle + j_range//2)

    i_slice = slice(i_middle - i_range//2 + padding_i, i_middle + i_range//2 + padding_i, None)
    j_slice = slice(j_middle - j_range//2 + padding_j, j_middle + j_range//2 + padding_j, None)

    return (i_slice, j_slice, cur_bbox[2])

In [26]:
def get_masked_nodules_pictures(frame_height=100, frame_width=100, black_mask=False):
    df_data = []
    for scan_index, scan in enumerate(scan_list):
        print(f"Current patient: {scan.patient_id} | {scan_index+1}/{len(scan_list)}")
        for ann_index, annotation in enumerate(scan.annotations):
            bbox = get_zoomed_out_bbox(annotation.bbox(), frame_height, frame_width)
            vol = annotation.scan.to_volume(verbose=False)
            z = math.floor(vol[bbox].shape[2] / 2) 
            masked_image = np.array(vol[bbox][:,:,z])
            
            if black_mask:
                bbox_np = np.array([[row.start, row.stop] for row in bbox])
                mask = annotation.boolean_mask(bbox=bbox_np)
                mask_map = np.array(mask[:,:,z])
                # modifying the original image such that:
                #     if mask_map[i][j] == False --> image[i][j] = black
                #     if mask_map[i][j] == True  --> image[i][j] is preserved
                for i in range(len(mask_map)-1):
                    for j in range(len(mask_map[0])-1):
                        if mask_map[i][j] == False:
                            masked_image[i][j] = -1000 # black color value

            # save image data and path
            zoomed_image_path = f"masked_images/imgseg-{scan.patient_id}-{ann_index+1}.png"
            mpimg.imsave(zoomed_image_path, masked_image, cmap="gray")
            image_data = cv2.imread(zoomed_image_path)
            image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
            image_data = np.array(image_data, dtype=np.float64)/255
            df_data.append({"Id": f"{scan.patient_id}-{ann_index+1}", "Zoomed_Image_Data": image_data})
    
    with open("masked_images_dict.pkl", "wb") as save_df:
        pickle.dump(df_data, save_df)
    
    df = pd.DataFrame(data=df_data, columns=["Id", "Zoomed_Image_Data"])
    df.to_csv("cnn_images_data.csv", sep=',')
    return df
                            

In [7]:
height, width = get_max_frame_size()
print(f"The largest border is [{width}]x[{height}] pixels.")

At scan 1/1018
At scan 2/1018
At scan 3/1018
At scan 4/1018
At scan 5/1018
At scan 6/1018
At scan 7/1018
At scan 8/1018
At scan 9/1018
At scan 10/1018
At scan 11/1018
At scan 12/1018
At scan 13/1018
At scan 14/1018
At scan 15/1018
At scan 16/1018
At scan 17/1018
At scan 18/1018
At scan 19/1018
At scan 20/1018
At scan 21/1018
At scan 22/1018
At scan 23/1018
At scan 24/1018
At scan 25/1018
At scan 26/1018
At scan 27/1018
At scan 28/1018
At scan 29/1018
At scan 30/1018
At scan 31/1018
At scan 32/1018
At scan 33/1018
At scan 34/1018
At scan 35/1018
At scan 36/1018
At scan 37/1018
At scan 38/1018
At scan 39/1018
At scan 40/1018
At scan 41/1018
At scan 42/1018
At scan 43/1018
At scan 44/1018
At scan 45/1018
At scan 46/1018
At scan 47/1018
At scan 48/1018
At scan 49/1018
At scan 50/1018
At scan 51/1018
At scan 52/1018
At scan 53/1018
At scan 54/1018
At scan 55/1018
At scan 56/1018
At scan 57/1018
At scan 58/1018
At scan 59/1018
At scan 60/1018
At scan 61/1018
At scan 62/1018
At scan 63/1018
A

In [32]:
width = 86
height = 98

print(f"The images will have size [{width}]x[{height}] pixels")

cnn_images_df = get_masked_nodules_pictures(height, width)
cnn_images_df

The images will have size [86]x[98] pixels
Current patient: LIDC-IDRI-0078 | 1/1018
Current patient: LIDC-IDRI-0069 | 2/1018
Current patient: LIDC-IDRI-0079 | 3/1018
Current patient: LIDC-IDRI-0101 | 4/1018
Current patient: LIDC-IDRI-0110 | 5/1018
Current patient: LIDC-IDRI-0115 | 6/1018
Current patient: LIDC-IDRI-0132 | 7/1018
Current patient: LIDC-IDRI-0136 | 8/1018
Current patient: LIDC-IDRI-0150 | 9/1018
Current patient: LIDC-IDRI-0151 | 10/1018
Current patient: LIDC-IDRI-0154 | 11/1018
Current patient: LIDC-IDRI-0001 | 12/1018
Current patient: LIDC-IDRI-0002 | 13/1018
Current patient: LIDC-IDRI-0003 | 14/1018
Current patient: LIDC-IDRI-0004 | 15/1018
Current patient: LIDC-IDRI-0005 | 16/1018
Current patient: LIDC-IDRI-0006 | 17/1018
Current patient: LIDC-IDRI-0007 | 18/1018
Current patient: LIDC-IDRI-0008 | 19/1018
Current patient: LIDC-IDRI-0009 | 20/1018
Current patient: LIDC-IDRI-0010 | 21/1018
Current patient: LIDC-IDRI-0011 | 22/1018
Current patient: LIDC-IDRI-0012 | 23/1018


,Id,Zoomed_Image_Data
0,LIDC-IDRI-0078-1,[[0.63921569 0.64705882 0.64705882 ... 0.56078...
1,LIDC-IDRI-0078-2,[[0.96470588 0.98431373 0.97254902 ... 0.08235...
2,LIDC-IDRI-0078-3,[[0.69019608 0.67843137 0.68235294 ... 0.66274...
3,LIDC-IDRI-0078-4,[[0.76078431 0.74901961 0.76078431 ... 0.65490...
4,LIDC-IDRI-0078-5,[[0.63921569 0.64705882 0.64705882 ... 0.56078...
...,...,...
6854,LIDC-IDRI-0638-1,[[0.08235294 0.07058824 0.05882353 ... 0.13725...
6855,LIDC-IDRI-0638-2,[[0.68627451 0.68235294 0.67843137 ... 0.69411...
6856,LIDC-IDRI-0638-3,[[0.68235294 0.67843137 0.68627451 ... 0.68235...
6857,LIDC-IDRI-0127-1,[[0.53333333 0.5372549 0.52156863 ... 0.02745...
